In [ ]:
from trello import TrelloClient
import requests

Hay varios approachs para manejar Trello. Su API + requests, un wrapper y add-ons.
Por simplicidad, use wrapper + requests

In [ ]:
# NO SUBIR KEY Y TOKEN A GITHUB
# opcion: poner como variable de entorno

# https://trello.com/app-key
TRELLO_API_KEY = ""
# Token
TRELLO_API_TOKEN = ""

# En que lista quieres revisar si hay cards que deberian volver a sus Backlogs ?
buscada = "Sugerencia Hacer en Este Sprint"

# que label le quieres poner cuando la movamos ?
label = "triage"

In [ ]:
# inicializamoss un client con nuestra key y token para acceder a los recursos
client = TrelloClient(
        api_key=TRELLO_API_KEY,
        api_secret=TRELLO_API_TOKEN)

In [ ]:
# Extraer boards
all_boards = client.list_boards()

In [ ]:
# Elegir board, aca yo saco el -1, pero se puede filtrar por nombre
last_board = all_boards[-1]
lists = last_board.list_lists()

In [ ]:
# buscamos label que queremos agregar. Se asume que solo hay un label con esa palabra
# Ojo que si tienen nombres duplicados podria fallar !

label = list(filter(lambda x: "triage" in x.name,last_board.get_labels()))[0]

In [ ]:
# Cuales son todas mis listas ?
print([board_list for board_list in lists])     

In [ ]:
# Para cada una de las listas de arriba, muestrame sus cartas
def imprimir(_list):
    cards = _list.list_cards()
    if cards:   
        print("\nLa lista {0} tiene las siguientes cards:".format(_list))
        [print("- ",card.name) for card in cards]
    else:
        print("\n\033[1mLa lista {0} no tiene cards.\033[0m".format(_list)) 
    
[imprimir(x) for x in lists]

In [ ]:
# filtramos la lista en la cual queremos revisar si hay cards que pertenecieron a backlogs

print("Cuando buscamos especificamente en la lista \033[1m'{0}'\033[0m sus cartas son:".format(buscada))
filtered_list = list(filter(lambda x: x.name == buscada, lists))
cards = filtered_list[0].list_cards()   # sacamos sus cartas
print(cards)

In [ ]:
# recibe lista de cartas, revisa sus movements, busca el Backlog más reciente al que pertenecio (si es que), 
# lo mueve de list, lo pone en primera posicion y le agrega el label

def mover_card(card, last_backlog_was, label):
    card.change_list(last_backlog_was)
    try:
        card.add_label(label)
    except:
        print("Ya tenia ese label (o hubo un error distinto)")
    card.change_pos(0)
    
def buscar_ultimo_backlog(movements):
    if len(movements) == 0:
        return None
    return [movement["source"]["id"] for movement in movements if "Backlog" in movement["source"]["name"]]
    return None
            
def revisar_cartas(cards):
    if len(cards) != 0:
        [mover_card(card, buscar_ultimo_backlog(card.list_movements())[0], label) for card in cards if buscar_ultimo_backlog(card.list_movements())]            
revisar_cartas(cards)